In [1]:
from mmdet.apis import init_detector
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed


import json

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import glob as _glob
import os
    
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches


In [3]:
#### load_annotations에서 뒤의 변수 받는거 custom dataset 에서는 이름을 바꿔도 되지만 아래에
#### configuration에서는 무조건 변수명을 ann_file로 받아야함
@DATASETS.register_module(name='Drive_dataset')
class Drive_dataset(CustomDataset):
    CLASSES=('car','bus','truck', 'special vehicle', 'motorcycle','bicycle','personal mobility','person','Traffic_light', 'Traffic_sign')


    def load_annotations(self, ann_fol):
        
        CLASSES_dict = {'car' : 0 , 'bus' : 1, 'truck' : 2, 'special vehicle' : 3, 'motorcycle' : 4,'bicycle' : 5 ,'personal mobility' : 6 
                        ,'person' : 7 ,'Traffic_light' : 8, 'Traffic_sign' : 9}
        
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        
        data_infos = []
        
        ls = glob(ann_fol,'*',True)
        
        for idx,an in enumerate(ls):
            json_data = {}
            with open(an, "r") as json_file:
                json_data = json.load(json_file)
                
            ansplit = an.split('/')
            
            filename = ansplit[0] + '/' + ansplit[1] + '/' + 'images'+'/'+ json_data['image_name']
            
            width, height = json_data['image_size']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for ann_data in json_data['Annotation']:
                gt_labels.append(CLASSES_dict[ann_data['class_name']])
                gt_bboxes.append(ann_data['data'])


            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(ls))+' load annotations END!')
            
        
        
        return data_infos

In [4]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('UniverseNet/configs/waymo_open/universenet50_2008_fp16_4x4_mstrain_640_1280_1x_waymo_open_f0.py') 

In [5]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
   

In [6]:
## 추가 및 수정 ## 
cfg.dataset_type  = 'Drive_dataset'
cfg.data_root = ''

## single GPU 이기 때문에 syncBN 이 아닌 BN으로 수정)
cfg.model.backbone.norm_cfg=dict(type='BN', requires_grad=True)

## 기존 UniverseNet에서 부른거 올려두고 4070 suepr가 감당할 수 있는 수준으로 수정하기
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(960, 640), (1920, 1200)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(
        type='PhotoMetricDistortion',
        brightness_delta=32,
        contrast_range=(0.5, 1.5),
        saturation_range=(0.5, 1.5),
        hue_delta=18),
     dict(
        type='RandomCrop',
        crop_type='absolute_range',
        crop_size=(640, 960)),  ##### crop size는 (height, width) 로 되있음
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
    
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
                type='MultiScaleFlipAug',
                img_scale=(1920, 1200),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
]

cfg.data=dict(
    samples_per_gpu=10,
    workers_per_gpu=12,
    train=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/training/labels/',
        pipeline=cfg.train_pipeline),
     val=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/validation/labels/',
        pipeline=cfg.test_pipeline),
    # test 데이터 
    test=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/validation/labels/',
        pipeline=cfg.test_pipeline),)

cfg.model.bbox_head.num_classes=10

cfg.device='cuda'
cfg.work_dir = 'checkpoints_ver1'

cfg.log_config.interval = 4000 #iteration 단위

cfg.seed = 2024

set_random_seed(cfg.seed, deterministic=False)

cfg.workflow = [('train', 1), ('val', 1)]

cfg.evaluation = dict(interval=1, metric='mAP')

cfg.load_from = 'universenet50_2008_fp16_4x4_mstrain_480_960_2x_coco_20200815_epoch_24-81356447.pth'
cfg.runner = dict(type='EpochBasedRunner', max_epochs=24)

cfg.gpu_ids = range(1)

In [7]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
       

In [8]:
from mmdet.datasets import build_dataset, build_dataloader

In [9]:
test_dataloader_default_args = dict(
    samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)

dataset = build_dataset(cfg.data.test)

test_loader_cfg = {
        **test_dataloader_default_args,
        **cfg.data.get('test_dataloader', {})
    }
# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, **test_loader_cfg)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/root/UniverseNet/mmdet/datasets/custom.py:186: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')


In [10]:
from mmdet.models import build_detector
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)

In [13]:
checkpoint_path = 'checkpoints_ver1/epoch_24.pth'

# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES

2024-08-16 06:25:41,718 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.0 is upgraded to version 2.
2024-08-16 06:25:41,719 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.1 is upgraded to version 2.
2024-08-16 06:25:41,719 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.2 is upgraded to version 2.
2024-08-16 06:25:41,722 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.0 is upgraded to version 2.
2024-08-16 06:25:41,722 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.1 is upgraded to version 2.
2024-08-16 06:25:41,723 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.2 is upgraded to version 2.
2024-08-16 06:25:41,725 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.0 is upgraded to version 2.
2024-08-16 06:25:41,725 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.1 is upgraded to version 2.
2024-08-16 06:25:41,726 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.2 is upgraded to version 2.


load checkpoint from local path: checkpoints_ver1/epoch_24.pth


In [14]:
from mmdet.utils import (build_ddp, build_dp, compat_cfg, get_device,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)

from mmdet.apis import single_gpu_test ## 4070 super 단독 환경

In [15]:
model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

[                               ] 2/10000, 3.0 task/s, elapsed: 1s, ETA:  3319s

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.6 task/s, elapsed: 643s, ETA:     0s

In [16]:
mAP_output = dataset.evaluate(outputs,metric='mAP',iou_thr=0.5)


---------------iou_thr: 0.5---------------

+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32964 | 257441 | 0.988  | 0.961 |
| bus               | 979   | 12413  | 0.968  | 0.921 |
| truck             | 5874  | 81709  | 0.979  | 0.921 |
| special vehicle   | 202   | 7959   | 0.891  | 0.724 |
| motorcycle        | 235   | 5264   | 0.834  | 0.732 |
| bicycle           | 62    | 1401   | 0.887  | 0.801 |
| personal mobility | 43    | 1724   | 0.930  | 0.780 |
| person            | 7347  | 59510  | 0.928  | 0.848 |
| Traffic_light     | 3298  | 22330  | 0.969  | 0.917 |
| Traffic_sign      | 2889  | 33124  | 0.968  | 0.906 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.851 |
+-------------------+-------+--------+--------+-------+
